In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carga de datos
df_life = pd.read_csv('lifeExpectancyAtBirth.csv')
df_medicos = pd.read_csv('medicos.csv')
df_paises = pd.read_csv('paises.csv')

# Tareas a realizar

1. Crea cada gráfico con Matplotlib de la forma más completa posible, añadiendo nombres de ejes, títulos, leyenda, etc. en función de los apuntes o lo que investigues por tu cuenta. Si usas una función o parámetro no visto en los apuntes, documéntala en el código con comentarios de línea. 
2. Una vez hecho el gráfico con Matplotlib, con ayuda de tu IA de confianza, pide que te covierta el gráfico a otro equivalente, pero más estiloso, usando la librería Seaborn. Documenta en el código para qué sirve cada función y parámetro usado en esta versión.

Cuando solamente se pide una gráfica puedes usar el enfoque orientado a función o invocar a subplots y trabajar a partir de ax.

# Gráfica 1

Del fichero **lifeExpectancyAtBirth.cs** muestra en un gráfico de línea la evolución de España, siendo el eje Y la esperanza de vida, el eje X el año de medición.

In [ ]:
df_spain = df_life[(df_life['Location'] == 'Spain') & (df_life['Dim1'] == 'Both sexes')]
plt.figure(figsize=(10, 5))
plt.plot(df_spain['Period'], df_spain['First Tooltip'], marker='o', color='red', linestyle='--')

plt.title('Evolución de la Esperanza de Vida en España')
plt.xlabel('Año')
plt.ylabel('Esperanza de vida (años)')
plt.grid(True, alpha=0.3)
plt.show()

# Gráfica 2

Del fichero **paises.csv**. Muestra en un gráfico de tarta el PIB de los paises de Europa, pero solamente los 10 paises con mayor PIB. Hazlo de manera que se vea también el % del PIB en el gráfico.

In [ ]:
top_10_pib_europa = df_paises[df_paises['continente'] == 'Europe'].nlargest(10, 'PIB')

plt.figure(figsize=(10, 8))

# labels: nombres de los países
# autopct: formato del porcentaje (%.1f%% indica un decimal y el símbolo %)
# startangle: gira el gráfico para una mejor visualización
plt.pie(top_10_pib_europa['PIB'], 
        labels=top_10_pib_europa['pais'], 
        autopct='%.1f%%', 
        startangle=140, 
        colors=plt.cm.Paired.colors)

plt.title('Top 10 Países con mayor PIB en Europa')
plt.axis('equal')  # Asegura que el gráfico sea un círculo perfecto
plt.show()

# Gráfica 3.

Haz lo mismo que en el gráfico 2 pero a nivel mundial, agrupando por Continente.

In [ ]:
df_mundial = pd.merge(df_life, df_paises, left_on='Location', right_on='pais')

# Filtramos para tener datos únicos por país (Año 2019 y ambos sexos)
df_2019 = df_mundial[(df_mundial['Period'] == 2019) & (df_mundial['Dim1'] == 'Both sexes')]

# Agrupamos por continente y calculamos la media
vida_por_continente = df_2019.groupby('continente')['First Tooltip'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))

# Creamos las barras
plt.bar(vida_por_continente.index, vida_por_continente.values, color='skyblue', edgecolor='navy')

# Añadimos detalles
plt.title('Esperanza de Vida Media por Continente (Año 2019)', fontsize=14)
plt.xlabel('Continente', fontsize=12)
plt.ylabel('Años de vida (Media)', fontsize=12)
plt.ylim(50, 85) # Ajustamos el eje Y para apreciar mejor las diferencias
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

# Gráfica 4.

1. De la unión interna de paises.csv y médicos.csv, agrupando por región, haz una nube de puntos con el PIB en el eje X y la cantidad de médicos en el eje Y.

2. Calcula la media mundial de médicos y del PIB.
   
4. Pinta sobre la gráfica el punto medio de cada eje, de otro color. Incluye la leyenda.

In [ ]:
# 1. Unión interna y agrupación por región
# Usamos la media de médicos (First Tooltip) y el PIB por región
df_merged = pd.merge(df_paises, df_medicos, left_on='pais', right_on='Location')
df_region = df_merged.groupby('region')[['PIB', 'First Tooltip']].mean()

# 2. Media mundial de médicos y PIB (calculada sobre el dataframe unido)
media_pib_mundial = df_merged['PIB'].mean()
media_medicos_mundial = df_merged['First Tooltip'].mean()

print(f"Media Mundial - PIB: {media_pib_mundial:.2f}, Médicos: {media_medicos_mundial:.2f}")

plt.figure(figsize=(12, 7))

# Nube de puntos por región
plt.scatter(df_region['PIB'], df_region['First Tooltip'], 
            color='royalblue', s=100, label='Regiones', alpha=0.7)

# 4. Pinta el punto medio mundial
plt.scatter(media_pib_mundial, media_medicos_mundial, 
            color='red', s=150, marker='X', label='Media Mundial')

# Estética
plt.title('Relación PIB vs Médicos por Región', fontsize=14)
plt.xlabel('PIB (Promedio por región)')
plt.ylabel('Médicos por cada 10.000 hab. (Promedio)')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()

plt.show()

# Gráfica 5. 

Haz lo mismo que en gráfica 4 pero desglosado por continente. 

Para ello debemos crear la gráfica a partir de subplots. Cada continente es un subplot diferente, debemos hacer 2 columnas y 3 filas, cada continente es un subplot.

In [ ]:
# Unión de datos
df_merged = pd.merge(pd.read_csv('paises.csv'), pd.read_csv('medicos.csv'), left_on='pais', right_on='Location')

# Obtenemos la lista de continentes únicos
continentes = df_merged['continente'].unique()

# Creamos una figura con 3 filas y 2 columnas
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 18))
fig.suptitle('Relación PIB vs Médicos por Continente', fontsize=20)

# Aplanamos el array de axes para iterar fácilmente (de 2D a 1D)
axes = axes.flatten()

for i, continente in enumerate(continentes):
    # Filtrar datos por continente
    df_cont = df_merged[df_merged['continente'] == continente]
    
    # Dibujar en el subplot correspondiente
    axes[i].scatter(df_cont['PIB'], df_cont['First Tooltip'], alpha=0.6, color='royalblue')
    
    # Configuración de cada subplot
    axes[i].set_title(f'Continente: {continente}', fontsize=14)
    axes[i].set_xlabel('PIB')
    axes[i].set_ylabel('Médicos (por 10k hab.)')
    axes[i].grid(True, linestyle=':', alpha=0.6)

# Eliminar subplots vacíos si hay menos de 6 continentes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajusta para que el título no solape
plt.show()

# Gráfica 6.

A partir de la unión de ambos csv, redondea la cantidad de médicos al valor entero más cercano y haz un histograma para relacionar la cantidad de médicos en el eje X y la frecuencia/cantidad de países con esos médicos en el eje Y. 

In [ ]:
# Unión interna
df_union = pd.merge(df_paises, df_medicos, left_on='pais', right_on='Location')

# Redondeo al entero más cercano
df_union['medicos_entero'] = df_union['First Tooltip'].round().astype(int)

plt.figure(figsize=(10, 6))

# plt.hist crea el histograma
# bins=30 divide el rango de médicos en 30 intervalos
plt.hist(df_union['medicos_entero'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)

plt.title('Distribución de Médicos por cada 10.000 habitantes', fontsize=14)
plt.xlabel('Cantidad de Médicos (Redondeado)')
plt.ylabel('Frecuencia (Número de registros/países)')
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.show()

# Gráfica 7. 

Haz lo mismo que la gráfica 1 pero con 4 paises en el mismo gráfico, cada país un colo diferente. España, Alemania, Francia, Italia.

In [ ]:
# Lista de países y filtrado (solo ambos sexos)
paises_list = ['Spain', 'Germany', 'France', 'Italy']
df_comparativa = df_life[(df_life['Location'].isin(paises_list)) & (df_life['Dim1'] == 'Both sexes')]

plt.figure(figsize=(12, 6))

# Iteramos para pintar cada país
for pais in paises_list:
    # Filtramos los datos específicos de ese país
    datos_pais = df_comparativa[df_comparativa['Location'] == pais]
    # Ordenamos por año para que la línea no se cruce
    datos_pais = datos_pais.sort_values('Period')
    
    plt.plot(datos_pais['Period'], datos_pais['First Tooltip'], marker='o', label=pais)

# Configuración de etiquetas y diseño
plt.title('Comparativa de Esperanza de Vida: España vs Vecinos Europeos', fontsize=14)
plt.xlabel('Año (Period)')
plt.ylabel('Esperanza de Vida (Años)')
plt.legend(title="Países") # Muestra la leyenda con el nombre de cada país
plt.grid(True, alpha=0.3)

plt.show()